## Imports and creds

In [1]:
from google.oauth2 import service_account

In [2]:
# Mount drive - needed for GCS
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [3]:
root_path = '/content/drive/MyDrive/GitHub'

## Create Google bucket

To deploy a model to the API, we'll need to push it to Google Storage. To do that, we first need a bucket set up.

In [10]:
from google.cloud import storage


def create_bucket_class_location(bucket_name):
    """
    Create a new bucket in the US region with the coldline storage
    class
    """

    storage_client = storage.Client(project = 'grav-waves-358320')

    bucket = storage_client.bucket(bucket_name)
    bucket.storage_class = "COLDLINE"
    new_bucket = storage_client.create_bucket(bucket)

    print(
        "Created bucket {} in {} with storage class {}".format(
            new_bucket.name, new_bucket.location, new_bucket.storage_class
        )
    )
    return new_bucket

In [13]:
def list_buckets():
    """Lists all buckets."""

    storage_client = storage.Client(project = 'grav-waves-358320')
    buckets = storage_client.list_buckets()

    for bucket in buckets:
        print(bucket.name)

In [16]:
create_bucket_class_location("gw-models")

Created bucket gw-models in US with storage class COLDLINE


<Bucket: gw-models>

Let's check that the bucket shows up.

In [18]:
list_buckets()

gw-models


## Push model to GCS

In [20]:
def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    # The ID of your GCS bucket
    # bucket_name = "your-bucket-name"
    # The path to your file to upload
    # source_file_name = "local/path/to/file"
    # The ID of your GCS object
    # destination_blob_name = "storage-object-name"

    storage_client = storage.Client(project = 'grav-waves-358320')
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print(
        f"File {source_file_name} uploaded to {destination_blob_name}."
    )

In [52]:
upload_blob('gw-models', '/content/drive/MyDrive/GitHub/grav-waves/models/gw_lstm1/keras_metadata.pb', 'gw_lstm1/keras_metadata.pb')

File /content/drive/MyDrive/GitHub/grav-waves/models/gw_lstm1/keras_metadata.pb uploaded to gw_lstm1/keras_metadata.pb.


In [53]:
upload_blob('gw-models', '/content/drive/MyDrive/GitHub/grav-waves/models/gw_lstm1/saved_model.pb', 'gw_lstm1/saved_model.pb')

File /content/drive/MyDrive/GitHub/grav-waves/models/gw_lstm1/saved_model.pb uploaded to gw_lstm1/saved_model.pb.


In [54]:
upload_blob('gw-models', '/content/drive/MyDrive/GitHub/grav-waves/models/gw_lstm1/variables/variables.index', 'gw_lstm1/variables/variables.index')

File /content/drive/MyDrive/GitHub/grav-waves/models/gw_lstm1/variables/variables.index uploaded to gw_lstm1/variables/variables.index.


In [55]:
upload_blob('gw-models', '/content/drive/MyDrive/GitHub/grav-waves/models/gw_lstm1/variables/variables.data-00000-of-00001', 'gw_lstm1/variables/variables.data-00000-of-00001')

File /content/drive/MyDrive/GitHub/grav-waves/models/gw_lstm1/variables/variables.data-00000-of-00001 uploaded to gw_lstm1/variables/variables.data-00000-of-00001.


Let's try using the command line to get all the files recursively with a single command.

In [31]:
!ls drive/MyDrive/GitHub/grav-waves/models/gw_lstm1

gw_lstm1


In [48]:
!pwd

/content


In [49]:
!SAVED_MODEL_DIR=$(ls /content/drive/MyDrive/GitHub/grav-waves/models/gw_lstm1 | tail -1)

In [50]:
!echo $SAVED_MODEL_DIR

In [51]:
!gsutil cp -r $SAVED_MODEL_DIR gs://gw-models

CommandException: Wrong number of arguments for "cp" command.


I'll work on writing a recursive function to go through all the folders from the top folder later.

## Create AI Platform Prediction Model
The next step is to do this programmatically. However following [these instructions](https://cloud.google.com/ai-platform/prediction/docs/deploying-models#console) allowed for rapid [API endpoint creation and versioning](https://console.cloud.google.com/ai-platform/models/grav_waves_lstm1;region=us-central1/versions?project=grav-waves-358320)